In [38]:
import random
import torch
from torch import nn

In [39]:
def create_context_examples(word_pairs, k=3):
    """
    Formats data for in-context learning:
    Input: "sing1 -> plur1; sing2 -> plur2; ...; singN -> "
    Target: "plurN"
    """
    context = []
    for _ in range(k):
        s, p = random.choice(word_pairs)
        context.append(f"{s} -> {p}")
    
    test_singular, test_plural = random.choice(word_pairs)
    context.append(f"{test_singular} -> ")
    
    return " ; ".join(context), test_plural


In [40]:
def make_batches(dataset, batch_size):
    """Groups dataset items into batches of specified size"""
    batch = []
    for item in dataset:
        batch.append(item)
        if len(batch) == batch_size:
            yield batch
            batch = []
    if batch:  # Yield remaining items
        yield batch


In [41]:
class InContextTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=64, nhead=2, num_layers=2):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = nn.Parameter(torch.randn(1, 512, d_model))
        self.layers = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead, 128),
            num_layers
        )
        self.head = nn.Linear(d_model, vocab_size)
        
    def forward(self, x):
        x = self.embed(x) + self.pos_embed[:, :x.size(1)]
        x = self.layers(x)
        return self.head(x)


In [42]:
def train_cycle(model, dataset, epochs=10, batch_size=32):
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
    loss_fn = nn.CrossEntropyLoss(ignore_index=0)

    for epoch in range(epochs):
        random.shuffle(dataset)
        total_loss = 0
        for batch in make_batches(dataset, batch_size):
            inputs = torch.stack([ex[0] for ex in batch])
            targets = torch.stack([ex[1] for ex in batch])

            outputs = model(inputs)

            loss = loss_fn(outputs.view(-1, outputs.size(-1)), targets.view(-1))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            total_loss += loss.item()
        
        print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")


In [22]:
word_pairs = [
    # Regular plurals
    ("cat", "cats"),
    ("dog", "dogs"),
    ("car", "cars"),
    ("apple", "apples"),
    ("book", "books"),
    ("tree", "trees"),
    ("cup", "cups"),
    ("pen", "pens"),
    ("chair", "chairs"),
    ("table", "tables"),
    ("house", "houses"),
    ("phone", "phones"),
    ("shoe", "shoes"),
    ("bag", "bags"),
    ("door", "doors"),
    ("window", "windows"),
    ("computer", "computers"),
    ("student", "students"),
    ("teacher", "teachers"),
    ("doctor", "doctors"),
    ("friend", "friends"),
    ("child", "children"),
    ("person", "people"),
    ("man", "men"),
    ("woman", "women"),
    ("mouse", "mice"),
    ("goose", "geese"),
    ("tooth", "teeth"),
    ("foot", "feet"),
    ("fish", "fish"),
    ("sheep", "sheep"),
    ("deer", "deer"),
    ("cactus", "cacti"),
    ("focus", "foci"),
    ("fungus", "fungi"),
    ("nucleus", "nuclei"),
    ("syllabus", "syllabi"),
    ("analysis", "analyses"),
    ("crisis", "crises"),
    ("thesis", "theses"),
    ("phenomenon", "phenomena"),
    ("criterion", "criteria"),
    ("datum", "data"),
    ("bus", "buses"),
    ("box", "boxes"),
    ("fox", "foxes"),
    ("watch", "watches"),
    ("wish", "wishes"),
    ("dish", "dishes"),
    ("baby", "babies"),
    ("city", "cities"),
    ("party", "parties"),
    ("story", "stories"),
    ("berry", "berries"),
    ("family", "families"),
    ("country", "countries"),
    ("lady", "ladies"),
    ("boy", "boys"),
    ("toy", "toys"),
    ("key", "keys"),
    ("day", "days"),
    ("monkey", "monkeys"),
    ("leaf", "leaves"),
    ("wolf", "wolves"),
    ("knife", "knives"),
    ("life", "lives"),
    ("wife", "wives"),
    ("calf", "calves"),
    ("half", "halves"),
    ("loaf", "loaves"),
    ("scarf", "scarves"),
    ("chief", "chiefs"),
    ("roof", "roofs"),
    ("belief", "beliefs"),
    ("chef", "chefs"),
    ("photo", "photos"),
    ("piano", "pianos"),
    ("halo", "halos"),
    ("potato", "potatoes"),
    ("tomato", "tomatoes"),
    ("hero", "heroes"),
    ("echo", "echoes"),
    ("zero", "zeroes"),
    ("kangaroo", "kangaroos"),
    ("radio", "radios"),
    ("studio", "studios"),
    ("video", "videos"),
    ("zoo", "zoos"),
    ("bamboo", "bamboos"),
    ("cargo", "cargoes"),
    ("volcano", "volcanoes"),
    ("tornado", "tornadoes"),
    ("mosquito", "mosquitoes"),
    ("buffalo", "buffaloes"),
    ("domino", "dominoes"),
    ("torpedo", "torpedoes"),
    ("veto", "vetoes"),
    ("alumnus", "alumni"),
    ("alumna", "alumnae"),
    ("medium", "media"),
    ("memorandum", "memoranda"),
    ("appendix", "appendices"),
    ("index", "indices"),
    ("matrix", "matrices"),
    ("vertex", "vertices"),
    ("axis", "axes"),
    ("ox", "oxen"),
    ("quiz", "quizzes"),
    ("church", "churches"),
    ("match", "matches"),
    ("branch", "branches"),
    ("peach", "peaches"),
    ("lunch", "lunches"),
    ("sandwich", "sandwiches"),
    ("witch", "witches"),
    ("pass", "passes"),
    ("glass", "glasses"),
    ("class", "classes"),
    ("kiss", "kisses"),
    ("bus", "buses"),
    ("gas", "gases"),
    ("status", "statuses"),
    ("octopus", "octopuses"),
    ("virus", "viruses"),
    ("radius", "radii"),
    ("genius", "geniuses"),
    ("species", "species"),
    ("series", "series"),
    ("aircraft", "aircraft"),
    ("means", "means"),
    ("barracks", "barracks"),
    ("salmon", "salmon"),
    ("shrimp", "shrimp"),
    ("trout", "trout"),
    ("swine", "swine"),
    ("hovercraft", "hovercraft"),
    ("crossroads", "crossroads"),
    ("headquarters", "headquarters"),
]


In [44]:
word_pairs = []
with open('nouns.csv', 'r') as f:
    for line in f:
        parts = line.strip().split(',')
        if len(parts) >= 2:
            singular = parts[0].strip()
            plural = parts[1].strip()
            word_pairs.append((singular, plural))
            
            # Check if composed of only lowercase letters and spaces
            #if all(c.islower() or c.isspace() for c in singular) and all(c.islower() or c.isspace() for c in plural):
print(word_pairs[:5])

[('007', '007s'), ('0', '0s'), ('0-10-0', '0-10-0s'), ('0-10-2', '0-10-2s'), ('0-12-0', '0-12-0s')]


In [45]:
from collections import defaultdict

def build_vocab(word_pairs):
    special_tokens = ["<pad>", "<unk>"]
    tokens = set()
    for s, p in word_pairs:
        tokens.update(s)
        tokens.update(p)
        tokens.add("->")
        tokens.add(";")

    token_list = special_tokens + sorted(tokens)
    stoi = defaultdict(lambda: 1, {tok: i for i, tok in enumerate(token_list)})  # <unk> = 1
    itos = {i: tok for tok, i in stoi.items()}
    return stoi, itos

def tokenize(text, stoi):
    return [stoi[c] for c in text]

def detokenize(indices, itos):
    return ''.join(itos[i] for i in indices if i > 1)


In [46]:
def encode_dataset(word_pairs, stoi, k=3, max_len=128):
    dataset = []
    for _ in range(1000):  # number of examples
        context, target = create_context_examples(word_pairs, k=k)
        input_ids = tokenize(context, stoi)
        target_ids = tokenize(target, stoi)
        
        input_tensor = torch.tensor(input_ids + [0] * (max_len - len(input_ids)), dtype=torch.long)[:max_len]
        target_tensor = torch.tensor(target_ids + [0] * (max_len - len(target_ids)), dtype=torch.long)[:max_len]
        
        dataset.append((input_tensor, target_tensor))
    return dataset


In [47]:
def evaluate(model, word_pairs, stoi, itos, k=3, max_len=128):
    model.eval()
    correct = 0
    total = 20
    with torch.no_grad():
        for _ in range(total):
            context, target = create_context_examples(word_pairs, k=k)
            input_ids = tokenize(context, stoi)
            input_tensor = torch.tensor(input_ids + [0] * (max_len - len(input_ids)), dtype=torch.long)[:max_len].unsqueeze(0)

            output_logits = model(input_tensor)
            output_ids = output_logits.argmax(dim=-1)[0].tolist()

            predicted = detokenize(output_ids[len(input_ids):], itos)
            print(f"{context}{predicted.strip()} (Expected: {target})")
            if predicted.strip().startswith(target):
                correct += 1

    print(f"Accuracy: {correct}/{total}")
    model.train()


In [48]:
stoi, itos = build_vocab(word_pairs)
stoi

defaultdict(<function __main__.build_vocab.<locals>.<lambda>()>,
            {'<pad>': 0,
             '<unk>': 1,
             ' ': 2,
             '!': 3,
             '"': 4,
             '#': 5,
             '$': 6,
             '%': 7,
             '&': 8,
             "'": 9,
             '(': 10,
             ')': 11,
             '*': 12,
             '+': 13,
             '-': 14,
             '->': 15,
             '.': 16,
             '/': 17,
             '0': 18,
             '1': 19,
             '2': 20,
             '3': 21,
             '4': 22,
             '5': 23,
             '6': 24,
             '7': 25,
             '8': 26,
             '9': 27,
             ';': 28,
             '?': 29,
             'A': 30,
             'B': 31,
             'C': 32,
             'D': 33,
             'E': 34,
             'F': 35,
             'G': 36,
             'H': 37,
             'I': 38,
             'J': 39,
             'K': 40,
             'L': 41,
            

In [ ]:
stoi, itos = build_vocab(word_pairs)
dataset = encode_dataset(word_pairs, stoi, k=1, max_len=64)
model = InContextTransformer(vocab_size=len(stoi), d_model=128, nhead=8, num_layers=30)

train_cycle(model, dataset, epochs=100, batch_size=1024)
evaluate(model, word_pairs, stoi, itos, k=1, max_len=64)


/Users/thompalec/cs182/182Proj/.venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1, Loss: 10.0637
Epoch 2, Loss: 9.5569
Epoch 3, Loss: 9.0670
Epoch 4, Loss: 8.5869
Epoch 5, Loss: 8.1351
Epoch 6, Loss: 7.7380
Epoch 7, Loss: 7.4077
Epoch 8, Loss: 7.1311
Epoch 9, Loss: 6.9097
Epoch 10, Loss: 6.7320
Epoch 11, Loss: 6.5810
Epoch 12, Loss: 6.4626
Epoch 13, Loss: 6.3615
Epoch 14, Loss: 6.2793
Epoch 15, Loss: 6.2158
Epoch 16, Loss: 6.1627
Epoch 17, Loss: 6.1138
Epoch 18, Loss: 6.0771
Epoch 19, Loss: 6.0404
Epoch 20, Loss: 6.0092
earmark -> earmarks ; augmentation cystoplasty -> ssessessssssss (Expected: augmentation cystoplasties)
syce -> syces ; not dog -> sssssssesessspsssssssssssessessssssss (Expected: not dogs)
measurable space -> measurable spaces ; kilomegacycle -> sssssss (Expected: kilomegacycles)
vanga -> vangas ; caching proxy -> sessspsssssssssssessessssssss (Expected: caching proxies)
rekick -> rekicks ; catfall -> sssesessspsssssssssssessessssssss (Expected: catfalls)
penalty phase -> penalty phases ; orbiton -> sssssssessessssssss (Expected: orbitons)
v